In [0]:
%sql
-- Merges data from a source table into a target table by matching on a key.
-- Updates existing rows or inserts new ones if no match is found.


In [0]:
from pyspark.sql import *
from delta.tables import *

In [0]:
schema = 'locations_db'
table = 'countries'

In [0]:

countries_df = spark.read.table(f"{schema}.{table}")

***Merge Into combines Update, Delete, and Insert operations. You have a target table and will merge records from a source table based on certain conditions***

In [0]:
countries_df.limit(3).display()

COUNTRY_ID,NAME,NATIONALITY,COUNTRY_CODE,ISO_ALPHA2,CAPITAL,POPULATION,AREA_KM2,REGION_ID,SUB_REGION_ID,INTERMEDIATE_REGION_ID,ORGANIZATION_REGION_ID
1,Afghanistan,Afghan,AFG,AF,Kabul,3.8041754E7,652230,30,30,null,30
2,Albania,Albanian,ALB,AL,Tirana,2880917.0,28748,20,70,null,20
3,Algeria,Algerian,DZA,DZ,Algiers,4.3053054E7,2381741,50,40,null,20


In [0]:
countries_1 = countries_df.filter("region_id in (10,20,30)")

In [0]:
countries_1.limit(3).display()

COUNTRY_ID,NAME,NATIONALITY,COUNTRY_CODE,ISO_ALPHA2,CAPITAL,POPULATION,AREA_KM2,REGION_ID,SUB_REGION_ID,INTERMEDIATE_REGION_ID,ORGANIZATION_REGION_ID
1,Afghanistan,Afghan,AFG,AF,Kabul,3.8041754E7,652230,30,30,null,30
2,Albania,Albanian,ALB,AL,Tirana,2880917.0,28748,20,70,null,20
5,Andorra,Andorran,AND,AD,Andorra la Vella,77142.0,468,20,70,null,20


In [0]:
countries_2 = countries_df.filter("region_id in (20,30,40,50)")

In [0]:
countries_2.limit(3).display()

COUNTRY_ID,NAME,NATIONALITY,COUNTRY_CODE,ISO_ALPHA2,CAPITAL,POPULATION,AREA_KM2,REGION_ID,SUB_REGION_ID,INTERMEDIATE_REGION_ID,ORGANIZATION_REGION_ID
1,Afghanistan,Afghan,AFG,AF,Kabul,3.8041754E7,652230,30,30,null,30
2,Albania,Albanian,ALB,AL,Tirana,2880917.0,28748,20,70,null,20
3,Algeria,Algerian,DZA,DZ,Algiers,4.3053054E7,2381741,50,40,null,20



**Save dataframes as delta tables in our location database.**

In [0]:
countries_1.write.mode('overwrite').saveAsTable('locations_db.countries_1')
countries_2.write.mode('overwrite').saveAsTable('locations_db.countries_2')

In [0]:
%sql
update countries_2 
  SET name = upper(name)

num_affected_rows
192


In [0]:
%sql
SELECT
  *
FROM
  countries_2
limit 5

COUNTRY_ID,NAME,NATIONALITY,COUNTRY_CODE,ISO_ALPHA2,CAPITAL,POPULATION,AREA_KM2,REGION_ID,SUB_REGION_ID,INTERMEDIATE_REGION_ID,ORGANIZATION_REGION_ID
1,AFGHANISTAN,Afghan,AFG,AF,Kabul,3.8041754E7,652230,30,30,null,30
2,ALBANIA,Albanian,ALB,AL,Tirana,2880917.0,28748,20,70,null,20
3,ALGERIA,Algerian,DZA,DZ,Algiers,4.3053054E7,2381741,50,40,null,20
4,AMERICAN SAMOA,American Samoan,ASM,AS,Pago Pago,55312.0,199,40,20,null,30
5,ANDORRA,Andorran,AND,AD,Andorra la Vella,77142.0,468,20,70,null,20



## Merge Into - SQL

In [0]:
%sql
SELECT current_database();

current_schema()
locations_db


In [0]:
%sql

MERGE INTO countries_1 AS tgt
USING countries_2 AS src
ON tgt.COUNTRY_ID = src.COUNTRY_ID

WHEN MATCHED THEN
  UPDATE SET
    tgt.name = src.name

WHEN NOT MATCHED THEN
  INSERT (
    tgt.country_id,
    tgt.name,
    tgt.nationality,
    tgt.country_code,
    tgt.iso_alpha2,
    tgt.capital,
    tgt.population,
    tgt.area_km2,
    tgt.region_id,
    tgt.sub_region_id,
    tgt.intermediate_region_id,
    tgt.organization_region_id
  )
  VALUES (
    src.country_id,
    src.name,
    src.nationality,
    src.country_code,
    src.iso_alpha2,
    src.capital,
    src.population,
    src.area_km2,
    src.region_id,
    src.sub_region_id,
    src.intermediate_region_id,
    src.organization_region_id
  );


num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
192,102,0,90


In [0]:
%sql
select * from countries_1

where
    region_id in (10,50)
order by iso_alpha2

limit 10

COUNTRY_ID,NAME,NATIONALITY,COUNTRY_CODE,ISO_ALPHA2,CAPITAL,POPULATION,AREA_KM2,REGION_ID,SUB_REGION_ID,INTERMEDIATE_REGION_ID,ORGANIZATION_REGION_ID
9,Antigua and Barbuda,Antiguan or Barbudan,ATG,AG,St. John's,97118.0,442,10,10,60,40
7,Anguilla,Anguillan,AIA,AI,The Valley,14869.0,91,10,10,60,40
6,ANGOLA,ANGOLAN,AGO,AO,Luanda,3.1825295E7,1246700,50,160,80,20
10,Argentina,Argentine,ARG,AR,Buenos Aires,4.4780677E7,2780400,10,10,40,40
12,Aruba,Aruban,ABW,AW,Oranjestad,106314.0,180,10,10,60,40
19,Barbados,Barbadian,BRB,BB,Bridgetown,287025.0,430,10,10,60,40
35,BURKINA FASO,BURKINAB�,BFA,BF,Ouagadougou,2.0321378E7,272967,50,160,10,20
36,BURUNDI,BURUNDIAN,BDI,BI,Bujumbura,1.153058E7,27834,50,160,50,20
23,BENIN,BENINESE,BEN,BJ,Porto-Novo,1.1801151E7,112622,50,160,10,20
184,Saint Barth�lemy,Barth�lemois,BLM,BL,Gustavia,9847.0,21,10,10,60,40
